# Hkl summarize and visualize
Investigate the completeness and redundancy of a single hkl file and produce a .res file for visualization.

### Steps for statistics summary:
1) Copy .hkl file to be investigated to Jupyter. If running via binder, individual files are stored in private session and not published online.
2) Fill all the necessary variables in the cell directly below these one.
    1) `hkl_path` and `cif_path` should be the filenames of copied files.
    2) For a full list of handled `hkl_format`s, see [hikari documentation](https://hikari.readthedocs.io/en/stable/autoapi/hikari/dataframes/hkl/index.html#hikari.dataframes.hkl.HklIo.format).
    3) If no cif file is provided, supply `hm_space_group_symbol` and `unit_cell` in Angstrom and degrees.
3) Run all cells in the notebook by selecting "Run > Run All cells" or repeating "Shift+Enter".
4) The table with final analysis results will be available in the last cell.

In [ ]:
# Variables to be modified
hkl_path = 'filename.hkl'
hkl_format = 'shelx_6'
cif_path = 'filename.cif'

# Provide variables below only if cif file has not been supplied
hm_space_group_symbol = 'P 1 21/n 1'
unit_cell = {'a': 10.0, 'b': 10.0, 'c': 10.0, 'al': 90, 'be': 120.0, 'ga': 90}

# After all variables are set, update them via `Ctrl+Enter`

In [ ]:
# Run this cell with `Ctrl+Enter` to get the evaluation
from hikari.dataframes import HklFrame, CifFrame
from hikari.symmetry import Group, PG, SG

h = HklFrame()
h.read(hkl_path, hkl_format)

try:
    c = CifFrame()
    c.read(cif_path)
    b = next(iter(c.values()))
    h.fill_from_cif_block(b)
    sg = Group.from_hall_symbol(b['_space_group_name_Hall'])
except FileNotFoundError:
    h.edit_cell(**unit_cell)
    sg = SG[hm_space_group_symbol.replace(' ', '')]

h.place()
h.stats(bins=10, space_group=sg)


### Following steps for hkl visualization in Mercury:
1) Run the cell below via "Ctrl+Enter" to create a convert the hkl file supplied before.
2) Download the newly-created `_hkl.res` and `hkl.msd` file
3) If visualizing a hkl file for the first time:
    1) In Mercury, open style manager by selecting "Display > Manage Styles..."
    2) "Open..." the new `hkl.msd` file to import a new style.
    3) Select the new "hkl" style, click "Apply" and restart Mercury to reset shaders
5) Open mercury and import the newly-created `_hkl.res` file.


In [ ]:
# Run this cell with `Ctrl+Enter` to get the visualization
from pathlib import Path

h_merged = h.copy()
h_merged.merge(point_group=PG['1'])
if 'm' not in h_merged.table:
    h_merged.table['m'] = 1
h_merged.to_res(path=Path(hkl_path).stem + '_hkl.res')